In [5]:
pip install --upgrade openpyxl bottleneck

  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for bottleneck from https://files.pythonhosted.org/packages/17/03/f89a2eff4f919a7c98433df3be6fd9787c72966a36be289ec180f505b2d5/bottleneck-1.5.0-cp311-cp311-win_amd64.whl.metadata
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ---------------------------------------- 0.0/112.1 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.1 kB ? eta -:--:--
   --------------------- ----------------- 61.4/112.1 kB 812.7 kB/s eta 0:00:01
   ---------------------------------------- 112.1/112.1 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled open

In [7]:
pip show openpyxl


Name: openpyxlNote: you may need to restart the kernel to use updated packages.

Version: 3.1.5
Summary: A Python library to read/write Excel 2010 xlsx/xlsm files
Home-page: https://openpyxl.readthedocs.io
Author: See AUTHORS
Author-email: charlie.clark@clark-consulting.eu
License: MIT
Location: C:\Users\Heber Castro\anaconda3\Lib\site-packages
Requires: et-xmlfile
Required-by: 


In [8]:
pip show bottleneck

Name: Bottleneck
Version: 1.5.0
Summary: Fast NumPy array functions written in C
Home-page: https://github.com/pydata/bottleneck
Author: 
Author-email: 
License: Simplified BSD
Location: C:\Users\Heber Castro\anaconda3\Lib\site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import time

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def buscar_nome_empresa_investsite(codigo_negociacao):
    """Busca o nome da empresa no site investsite.com.br."""
    if not codigo_negociacao or pd.isna(codigo_negociacao):
        logging.warning("buscar_nome_empresa_investsite: Código de negociação inválido ou ausente.")
        return None

    codigo_negociacao_fmt = str(codigo_negociacao).strip().upper()

    # URLs a serem tentadas (uma para ações, outra para FIIs, que são comuns)
    urls_para_tentar = [
        f"https://www.investsite.com.br/principais_acoes.php?cod_negociacao={codigo_negociacao_fmt}",
        f"https://www.investsite.com.br/fundos_imobiliarios_detalhes.php?cod_negociacao={codigo_negociacao_fmt}"
    ]

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for url_idx, url in enumerate(urls_para_tentar):
        logging.debug(f"Tentando URL ({url_idx+1}/{len(urls_para_tentar)}): {url} para o código {codigo_negociacao_fmt}")
        try:
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status() # Lança erro para códigos HTTP 4xx/5xx
            soup = BeautifulSoup(response.content, 'lxml')

            h1_tag = soup.find('h1')
            if h1_tag:
                full_text = h1_tag.get_text(separator=" ", strip=True)
                nome_empresa = full_text
                
                small_tag_text = ""
                if h1_tag.small:
                    small_tag_text = h1_tag.small.get_text(strip=True)
                    # Remove o texto da tag <small> do nome da empresa
                    nome_empresa = nome_empresa.replace(small_tag_text, "").strip()
                
                # Remove o próprio código de negociação do nome, caso ainda esteja lá
                nome_empresa = nome_empresa.replace(codigo_negociacao_fmt, "").strip()
                
                # Remove possíveis sufixos comuns que não fazem parte do nome principal
                # (Ex: "ON", "PN", "PNA", "UNT", "ED", "EJ", etc. - adicione mais se necessário)
                sufixos_comuns = [" ON", " PN", " PNA", " PNB", " UNT", " EDJ", " ED", " EJ"]
                for sufixo in sufixos_comuns:
                    if nome_empresa.upper().endswith(sufixo):
                        nome_empresa = nome_empresa[:-len(sufixo)]
                nome_empresa = nome_empresa.strip()


                if nome_empresa:
                    logging.info(f"Nome da empresa encontrado para {codigo_negociacao_fmt} em {url}: {nome_empresa}")
                    return nome_empresa
                else:
                    logging.warning(f"Nome extraído de <h1> resultou vazio para {codigo_negociacao_fmt} em {url}. Full text: '{full_text}'")
            else:
                logging.debug(f"Tag <h1> não encontrada em {url} para {codigo_negociacao_fmt}.")

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                logging.debug(f"Página não encontrada (404) em {url} para {codigo_negociacao_fmt}.")
            else:
                logging.warning(f"Erro HTTP ao buscar nome da empresa em {url} para {codigo_negociacao_fmt}: {e}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro de requisição ao buscar nome da empresa em {url} para {codigo_negociacao_fmt}: {e}")
            return None # Se houver erro de rede, provavelmente não adianta tentar outras URLs
        except Exception as e:
            logging.error(f"Erro inesperado ao processar {url} para {codigo_negociacao_fmt}: {e}")
        
        time.sleep(0.5) # Pequena pausa antes de tentar a próxima URL para o mesmo código

    logging.warning(f"Nome da empresa NÃO encontrado para {codigo_negociacao_fmt} após tentar todas as URLs.")
    return None

# --- Caminhos dos Arquivos ---
caminho_excel_origem = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas.xlsx"
caminho_excel_destino = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx" # Nome de arquivo ajustado

# --- Lógica Principal ---
def main():
    try:
        df = pd.read_excel(caminho_excel_origem)
        logging.info(f"Planilha '{caminho_excel_origem}' carregada com {len(df)} linhas.")
    except FileNotFoundError:
        logging.error(f"Arquivo Excel não encontrado em: {caminho_excel_origem}")
        return
    except Exception as e:
        logging.error(f"Erro ao carregar o arquivo Excel: {e}")
        return

    # Definição dos nomes das colunas conforme sua planilha
    COLUNA_CODIGO_NEGOCIACAO = "Codigo Negociacao"  # CORRIGIDO AQUI
    COLUNA_EMPRESA_NOME = "Empresa Capital Aberto"

    # Verificar se a coluna de código de negociação existe
    if COLUNA_CODIGO_NEGOCIACAO not in df.columns:
        logging.error(f"A coluna '{COLUNA_CODIGO_NEGOCIACAO}' é essencial e não foi encontrada na planilha.")
        logging.info(f"Colunas encontradas na planilha: {list(df.columns)}")
        return

    # Assegurar que a coluna de nome da empresa existe, caso contrário, criá-la
    if COLUNA_EMPRESA_NOME not in df.columns:
        df[COLUNA_EMPRESA_NOME] = pd.NA # Usar pd.NA para valores ausentes
        logging.info(f"Coluna '{COLUNA_EMPRESA_NOME}' não encontrada, foi criada.")


    total_linhas = len(df)
    nomes_preenchidos_nesta_execucao = 0
    nomes_ja_existiam = 0
    erros_busca_nome = 0
    codigos_processados = 0


    for index, row in df.iterrows():
        codigo_negociacao_original = row.get(COLUNA_CODIGO_NEGOCIACAO)

        if pd.isna(codigo_negociacao_original) or not str(codigo_negociacao_original).strip():
            logging.warning(f"Linha {index+2}: '{COLUNA_CODIGO_NEGOCIACAO}' ausente ou inválido. Pulando.")
            continue
        
        codigos_processados += 1
        codigo_negociacao_str = str(codigo_negociacao_original).strip().upper()
        
        # Atualizar o código na planilha para a versão normalizada (string, uppercase, sem espaços extras)
        # Isso garante consistência e ajuda na busca.
        # Usar .loc para atribuição para evitar SettingWithCopyWarning
        if df.loc[index, COLUNA_CODIGO_NEGOCIACAO] != codigo_negociacao_str:
            df.loc[index, COLUNA_CODIGO_NEGOCIACAO] = codigo_negociacao_str
            logging.debug(f"'{COLUNA_CODIGO_NEGOCIACAO}' normalizado para {codigo_negociacao_str} na linha {index+2}.")

        logging.info(f"Processando linha {index+2}/{total_linhas}: Código '{codigo_negociacao_str}'")

        nome_empresa_atual = row.get(COLUNA_EMPRESA_NOME)
        # Considera nome válido se for string e não estiver vazia após remover espaços
        nome_valido_existe = isinstance(nome_empresa_atual, str) and nome_empresa_atual.strip() != ""

        if not nome_valido_existe:
            logging.info(f"'{COLUNA_EMPRESA_NOME}' ausente ou inválido para '{codigo_negociacao_str}'. Buscando em InvestSite...")
            nome_empresa_web = buscar_nome_empresa_investsite(codigo_negociacao_str)
            if nome_empresa_web:
                df.loc[index, COLUNA_EMPRESA_NOME] = nome_empresa_web
                nomes_preenchidos_nesta_execucao += 1
            else:
                logging.warning(f"Não foi possível obter o nome da empresa para '{codigo_negociacao_str}' do InvestSite.")
                erros_busca_nome +=1
            time.sleep(1.2) # Aumentei um pouco a pausa para ser mais gentil com o servidor
        else:
            logging.info(f"'{COLUNA_EMPRESA_NOME}' já preenchido para '{codigo_negociacao_str}': {nome_empresa_atual}")
            nomes_ja_existiam +=1

    try:
        df.to_excel(caminho_excel_destino, index=False)
        logging.info(f"Planilha atualizada salva em: {caminho_excel_destino}")
        logging.info("--- Resumo da Execução ---")
        logging.info(f"Total de linhas na planilha: {total_linhas}")
        logging.info(f"Códigos de negociação válidos processados: {codigos_processados}")
        logging.info(f"Nomes de empresa preenchidos nesta execução: {nomes_preenchidos_nesta_execucao}")
        logging.info(f"Nomes de empresa que já existiam e foram mantidos: {nomes_ja_existiam}")
        logging.info(f"Códigos para os quais não foi possível encontrar nome na web: {erros_busca_nome}")

    except Exception as e:
        logging.error(f"Erro ao salvar a planilha atualizada: {e}")

if __name__ == "__main__":
    main()

2025-05-20 15:17:19,913 - INFO - Planilha 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas.xlsx' carregada com 48 linhas.
2025-05-20 15:17:19,915 - INFO - Processando linha 2/48: Código 'BRAP3, BRAP4'
2025-05-20 15:17:19,916 - INFO - 'Empresa Capital Aberto' já preenchido para 'BRAP3, BRAP4': BRADESPAR
2025-05-20 15:17:19,916 - INFO - Processando linha 3/48: Código 'VALE3'
2025-05-20 15:17:19,917 - INFO - 'Empresa Capital Aberto' já preenchido para 'VALE3': VALE
2025-05-20 15:17:19,917 - INFO - Processando linha 4/48: Código 'GGBR3,GGBR4'
2025-05-20 15:17:19,917 - INFO - 'Empresa Capital Aberto' já preenchido para 'GGBR3,GGBR4': GERDAU
2025-05-20 15:17:19,919 - INFO - Processando linha 5/48: Código 'CSNA3'
2025-05-20 15:17:19,919 - INFO - 'Empresa Capital Aberto' já preenchido para 'CSNA3': CIA_SIDERURGICA_NACIONAL
2025-05-20 15:17:19,919 - INFO - Processando linha 6/48: Código 'CBAV3'
2025-05-20 15:17:19,919 - INFO - 'Empresa Capital Aberto' já preenchido para 'CBAV3': CBA
2025-0

In [5]:
import pandas as pd

# --- Configuração dos Nomes dos Arquivos e Colunas ---

# Arquivos de referência
ARQUIVO_ANTERIOR = r"C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx"
COLUNA_CODIGO_ANTERIOR = "cod_negociacao" # Nome esperado do cabeçalho da coluna E
# COLUNA_NOME_ANTERIOR = "nome_empresa" # Nome esperado do cabeçalho da coluna G (não usado na comparação primária)

ARQUIVO_ATUAL = r"C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx" # Corrigido para .xlsx, se for .xls avise
COLUNA_CODIGO_ATUAL = "cod_negociacao"    # Nome esperado do cabeçalho da coluna E
# COLUNA_NOME_ATUAL = "nome_empresa"      # Nome esperado do cabeçalho da coluna G (não usado na comparação primária)

# Arquivo novo a ser verificado
ARQUIVO_NOVO_ATUALIZADO = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx" # Corrigido para .xlsx
COLUNA_CODIGO_NOVO = "Codigo Negociacao"
# COLUNA_NOME_NOVO = "Empresa Capital Aberto"

def carregar_codigos(filepath, column_name):
    """Carrega códigos de uma coluna específica de um arquivo Excel."""
    try:
        # Tenta ler apenas a coluna necessária para economizar memória/tempo
        df = pd.read_excel(filepath, usecols=[column_name])
        # Remove linhas onde o código é NaN, converte para string, remove espaços e põe em maiúsculas
        codigos = df[column_name].dropna().astype(str).str.strip().str.upper().unique()
        print(f"Carregados {len(codigos)} códigos únicos de '{filepath}' da coluna '{column_name}'.")
        return set(codigos)
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado: {filepath}")
        return set()
    except ValueError as ve:
        # Isso pode acontecer se a coluna especificada em usecols não existir
        print(f"ERRO: Coluna '{column_name}' não encontrada em '{filepath}' ou outro erro de valor: {ve}")
        # Tentar ler todas as colunas para inspecionar
        try:
            df_full = pd.read_excel(filepath)
            print(f"Colunas disponíveis em '{filepath}': {list(df_full.columns)}")
        except Exception as e_full:
            print(f"Não foi possível ler as colunas de '{filepath}': {e_full}")
        return set()
    except Exception as e:
        print(f"ERRO ao ler o arquivo '{filepath}': {e}")
        return set()

def main():
    print("Iniciando comparação de planilhas...")

    # 1. Carregar códigos dos arquivos de referência
    codigos_anterior = carregar_codigos(ARQUIVO_ANTERIOR, COLUNA_CODIGO_ANTERIOR)
    codigos_atual = carregar_codigos(ARQUIVO_ATUAL, COLUNA_CODIGO_ATUAL)

    if not codigos_anterior and not codigos_atual:
        print("Nenhum código carregado dos arquivos de referência. Abortando.")
        return

    # 2. Combinar códigos de referência
    codigos_referencia_total = codigos_anterior.union(codigos_atual)
    print(f"Total de {len(codigos_referencia_total)} códigos de referência únicos combinados.")

    if not codigos_referencia_total:
        print("Nenhum código de referência para comparar. Verifique os arquivos de origem.")
        return

    # 3. Carregar códigos do arquivo novo/atualizado
    codigos_novo_arquivo = carregar_codigos(ARQUIVO_NOVO_ATUALIZADO, COLUNA_CODIGO_NOVO)

    if not codigos_novo_arquivo:
        print(f"Nenhum código carregado do arquivo '{ARQUIVO_NOVO_ATUALIZADO}'. Não é possível comparar.")
        # Mesmo assim, podemos listar os que deveriam estar lá.
        print("\n--- Códigos de Referência que DEVERIAM estar no arquivo novo ---")
        if codigos_referencia_total:
            for i, codigo in enumerate(sorted(list(codigos_referencia_total))):
                print(f"{i+1}. {codigo}")
        else:
            print("Nenhum.")
        return


    # 4. Comparar: encontrar códigos de referência que estão faltando no arquivo novo
    codigos_faltando_no_novo = codigos_referencia_total.difference(codigos_novo_arquivo)

    # 5. (Opcional) Encontrar códigos no novo arquivo que não estão na referência
    codigos_extras_no_novo = codigos_novo_arquivo.difference(codigos_referencia_total)

    print("\n--- RESULTADO DA COMPARAÇÃO ---")

    if not codigos_faltando_no_novo:
        print(f"\nTODAS AS {len(codigos_referencia_total)} EMPRESAS DOS ARQUIVOS DE REFERÊNCIA FORAM ENCONTRADAS NO ARQUIVO '{ARQUIVO_NOVO_ATUALIZADO}'!")
    else:
        print(f"\nATENÇÃO: {len(codigos_faltando_no_novo)} códigos de referência NÃO foram encontrados em '{ARQUIVO_NOVO_ATUALIZADO}':")
        for i, codigo in enumerate(sorted(list(codigos_faltando_no_novo))):
            print(f"{i+1}. {codigo}")

    if codigos_extras_no_novo:
        print(f"\nINFO: {len(codigos_extras_no_novo)} códigos estão em '{ARQUIVO_NOVO_ATUALIZADO}' mas NÃO estavam nos arquivos de referência:")
        for i, codigo in enumerate(sorted(list(codigos_extras_no_novo))):
            print(f"{i+1}. {codigo}")
    else:
        print(f"\nINFO: Não há códigos em '{ARQUIVO_NOVO_ATUALIZADO}' que não estejam nos arquivos de referência.")

    print("\nComparação concluída.")

if __name__ == "__main__":
    main()

Iniciando comparação de planilhas...
Carregados 385 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx' da coluna 'cod_negociacao'.
Carregados 382 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx' da coluna 'cod_negociacao'.
Total de 400 códigos de referência únicos combinados.
Carregados 48 códigos únicos de 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx' da coluna 'Codigo Negociacao'.

--- RESULTADO DA COMPARAÇÃO ---

ATENÇÃO: 366 códigos de referência NÃO foram encontrados em 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx':
1. AALR3
2. AERI3
3. AESB3
4. AGRO3
5. AGXY3
6. ALLD3
7. ALPA3
8. ALPA4
9. ALPK3
10. ALSO3
11. ALUP11
12. ALUP3
13. ALUP4
14. AMAR3
15. AMBP3
16. AMER3
17. ANIM3
18. APER3
19. ARML3
20. ARZZ3
21. ASAI3
22. ATMP3
23. ATOM3
24. AVLL3
25. AZEV3
26. AZEV4
27. B3SA3
28. BAHI3
29. BALM3
30. BALM4
31. BBDC3
32. BBDC4
33. BBSE3
34. BDLL3
35. BDLL4


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import time
import re # Para expressões regulares (limpeza de nome)

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuração dos Nomes dos Arquivos e Colunas ---
ARQUIVO_ANTERIOR = r"C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx"
COLUNA_CODIGO_ANTERIOR = "cod_negociacao"

ARQUIVO_ATUAL = r"C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx"
COLUNA_CODIGO_ATUAL = "cod_negociacao"

# Arquivo NOVO a ser GERADO
ARQUIVO_FINAL_COMPLETO = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL.xlsx"
COLUNA_CODIGO_FINAL = "Codigo Negociacao"
COLUNA_NOME_FINAL = "Empresa Capital Aberto"


def carregar_e_desmembrar_codigos(filepath, column_name):
    """Carrega códigos, desmembra se estiverem agrupados, e normaliza."""
    codigos_finais = set()
    try:
        df = pd.read_excel(filepath, usecols=[column_name])
        raw_codigos = df[column_name].dropna().astype(str).str.strip().str.upper()
        
        for item_lista_codigos in raw_codigos:
            # Separa por vírgula, ponto e vírgula ou espaço, e remove espaços extras de cada parte
            partes = re.split(r'[,;\s]+', item_lista_codigos)
            for codigo_individual in partes:
                codigo_limpo = codigo_individual.strip()
                if codigo_limpo: # Adiciona apenas se não for uma string vazia
                    codigos_finais.add(codigo_limpo)
        
        print(f"Carregados e desmembrados {len(codigos_finais)} códigos únicos de '{filepath}' da coluna '{column_name}'.")
        return codigos_finais
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado: {filepath}")
        return set()
    except ValueError as ve:
        print(f"ERRO: Coluna '{column_name}' não encontrada em '{filepath}' ou outro erro de valor: {ve}")
        try:
            df_full = pd.read_excel(filepath)
            print(f"Colunas disponíveis em '{filepath}': {list(df_full.columns)}")
        except Exception as e_full:
            print(f"Não foi possível ler as colunas de '{filepath}': {e_full}")
        return set()
    except Exception as e:
        print(f"ERRO ao ler o arquivo '{filepath}': {e}")
        return set()

def buscar_nome_empresa_site(codigo_negociacao):
    """Busca o nome da empresa no site investsite.com.br ou infomoney."""
    if not codigo_negociacao or pd.isna(codigo_negociacao):
        logging.warning("buscar_nome_empresa_site: Código de negociação inválido ou ausente.")
        return None

    codigo_fmt = str(codigo_negociacao).strip().upper()
    urls_e_parsers = []

    # --- InvestSite ---
    investsite_urls = [
        f"https://www.investsite.com.br/principais_acoes.php?cod_negociacao={codigo_fmt}",
        f"https://www.investsite.com.br/fundos_imobiliarios_detalhes.php?cod_negociacao={codigo_fmt}"
    ]
    for url in investsite_urls:
        urls_e_parsers.append({
            "url": url,
            "parser_func": parse_investsite_nome,
            "site_name": "InvestSite"
        })
    
    # --- InfoMoney (Exemplo, a estrutura exata pode precisar de ajuste) ---
    # A URL do InfoMoney pode variar dependendo se é ação, FII, etc.
    # Este é um palpite, pode precisar de ajuste após inspecionar o site.
    # Exemplo para ações: https://www.infomoney.com.br/cotacoes/b3/acao/petrobras-petr4/
    # Exemplo para FIIs: https://www.infomoney.com.br/cotacoes/b3/fii/hglg11/
    # Para simplificar, vamos tentar um formato genérico, mas o InfoMoney é mais complexo para scraping direto assim.
    # InfoMoney pode ser mais difícil devido a JavaScript e estruturas dinâmicas.
    # Infelizmente, o InfoMoney é bem protegido contra scraping simples para dados de cotação/empresa.
    # A busca no InfoMoney pode ser mais complexa e exigir Selenium ou APIs (se disponíveis).
    # Por enquanto, vamos focar mais no InvestSite que é mais direto.

    # Exemplo de como poderia ser (MAS PROVAVELMENTE NÃO FUNCIONARÁ BEM PARA INFOMONEY SEM MAIS ANÁLISE):
    # infomoney_url = f"https://www.infomoney.com.br/cotacoes/{codigo_fmt.lower()}/" # URL muito genérica
    # urls_e_parsers.append({
    #     "url": infomoney_url,
    #     "parser_func": parse_infomoney_nome, # Precisaria criar esta função
    #     "site_name": "InfoMoney"
    # })


    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for item in urls_e_parsers:
        url = item["url"]
        parser_func = item["parser_func"]
        site_name = item["site_name"]

        logging.debug(f"Tentando {site_name} URL: {url} para o código {codigo_fmt}")
        try:
            response = requests.get(url, headers=headers, timeout=15)
            # Alguns sites podem retornar 200 mesmo para página não encontrada, mas com conteúdo indicando isso.
            # O raise_for_status é bom para erros claros 4xx/5xx.
            if response.status_code == 404:
                 logging.debug(f"Página não encontrada (404) em {site_name} para {codigo_fmt} na URL: {url}")
                 continue # Tenta próxima URL/site
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'lxml')
            nome_empresa = parser_func(soup, codigo_fmt)

            if nome_empresa:
                logging.info(f"Nome da empresa encontrado para {codigo_fmt} em {site_name}: {nome_empresa}")
                return nome_empresa
        
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404: # Tratado acima, mas bom ter aqui também.
                logging.debug(f"Página não encontrada (404) em {site_name} para {codigo_fmt} na URL: {url}")
            else:
                logging.warning(f"Erro HTTP ao buscar em {site_name} para {codigo_fmt} (URL: {url}): {e}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro de requisição ao buscar em {site_name} para {codigo_fmt} (URL: {url}): {e}")
            # Se for um erro de rede, pode não adiantar tentar outros sites imediatamente.
            # No entanto, como estamos iterando, vamos permitir que tente o próximo.
        except Exception as e:
            logging.error(f"Erro inesperado ao processar {site_name} para {codigo_fmt} (URL: {url}): {e}")
        
        time.sleep(0.5) # Pequena pausa antes de tentar a próxima URL/site

    logging.warning(f"Nome da empresa NÃO encontrado para {codigo_fmt} após tentar todos os sites/URLs.")
    return None

def parse_investsite_nome(soup, codigo_negociacao):
    """Extrai o nome da empresa da página do InvestSite."""
    h1_tag = soup.find('h1')
    if h1_tag:
        full_text = h1_tag.get_text(separator=" ", strip=True)
        nome_empresa = full_text
        
        small_tag_text = ""
        if h1_tag.small:
            small_tag_text = h1_tag.small.get_text(strip=True)
            nome_empresa = nome_empresa.replace(small_tag_text, "").strip()
        
        nome_empresa = nome_empresa.replace(codigo_negociacao, "").strip()
        
        sufixos_comuns = [" ON", " PN", " PNA", " PNB", " UNT", " EDJ", " ED", " EJ", " CI"] # Adicionado CI para FIIs
        # Remover sufixos de forma case-insensitive e apenas se forem palavras inteiras no final
        for sufixo in sufixos_comuns:
            # Regex para encontrar o sufixo no final da string, precedido por espaço ou início da string, case insensitive
            pattern = r"(?i)(\s*)" + re.escape(sufixo.strip()) + r"$"
            nome_empresa = re.sub(pattern, "", nome_empresa).strip()

        # Remover qualquer resquício do código de negociação que possa ter sobrado
        nome_empresa = re.sub(r'\b' + re.escape(codigo_negociacao) + r'\b', '', nome_empresa, flags=re.IGNORECASE).strip()
        # Limpar múltiplos espaços
        nome_empresa = re.sub(r'\s+', ' ', nome_empresa).strip()

        return nome_empresa if nome_empresa else None
    return None

# def parse_infomoney_nome(soup, codigo_negociacao):
#     """Extrai o nome da empresa da página do InfoMoney (PRECISA SER IMPLEMENTADO CORRETAMENTE)."""
#     # ESTA É UMA IMPLEMENTAÇÃO MUITO SIMPLISTA E PROVAVELMENTE NÃO FUNCIONARÁ
#     # O InfoMoney tem uma estrutura mais complexa.
#     # Você precisaria inspecionar o HTML de uma página de ativo no InfoMoney
#     # e encontrar o elemento que contém o nome da empresa.
#     title_tag = soup.find('title') # Exemplo muito básico
#     if title_tag:
#         title_text = title_tag.get_text()
#         # Tentar extrair o nome do título, pode ser algo como "PETR4: Cotação Ações Petrobras | InfoMoney"
#         # Precisaria de lógica mais robusta aqui.
#         parts = title_text.split(':')
#         if len(parts) > 1:
#             name_part = parts[1].split('|')[0].strip()
#             # Remover o código se estiver presente
#             name_part = name_part.replace(codigo_negociacao, "").strip()
#             return name_part
#     return None


def main():
    print("Iniciando processo de coleta e preenchimento de nomes de empresas...")

    # 1. Carregar e unificar todos os códigos de referência
    codigos_ref_anterior = carregar_e_desmembrar_codigos(ARQUIVO_ANTERIOR, COLUNA_CODIGO_ANTERIOR)
    codigos_ref_atual = carregar_e_desmembrar_codigos(ARQUIVO_ATUAL, COLUNA_CODIGO_ATUAL)
    
    todos_codigos_referencia = sorted(list(codigos_ref_anterior.union(codigos_ref_atual)))
    
    if not todos_codigos_referencia:
        print("Nenhum código de referência encontrado. Abortando.")
        return
    
    print(f"Total de {len(todos_codigos_referencia)} códigos de negociação únicos para processar.")

    # 2. Preparar dados para a nova planilha
    dados_finais = [] # Lista de dicionários para criar o DataFrame

    # Tentar carregar a planilha "Atualizado" se existir, para pegar nomes já existentes
    nomes_pre_existentes = {}
    arquivo_atualizado_original = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx"
    col_cod_atualizado = "Codigo Negociacao" # Conforme script anterior
    col_nome_atualizado = "Empresa Capital Aberto" # Conforme script anterior

    try:
        df_atualizado_orig = pd.read_excel(arquivo_atualizado_original)
        print(f"Carregada planilha '{arquivo_atualizado_original}' para verificar nomes pré-existentes.")
        for _, row in df_atualizado_orig.iterrows():
            cod_orig = row.get(col_cod_atualizado)
            nome_orig = row.get(col_nome_atualizado)

            if pd.notna(cod_orig) and pd.notna(nome_orig) and str(nome_orig).strip():
                # Desmembrar códigos da planilha atualizada também, se estiverem agrupados
                cod_orig_str = str(cod_orig).strip().upper()
                partes_cod_orig = re.split(r'[,;\s]+', cod_orig_str)
                for c in partes_cod_orig:
                    if c and c not in nomes_pre_existentes: # Adiciona apenas se não existir para evitar sobrescrita por nome de código agrupado
                         nomes_pre_existentes[c] = str(nome_orig).strip()
        print(f"Encontrados {len(nomes_pre_existentes)} nomes na planilha pré-existente.")

    except FileNotFoundError:
        print(f"Arquivo '{arquivo_atualizado_original}' não encontrado. Nomes serão buscados online para todos.")
    except Exception as e:
        print(f"Erro ao ler '{arquivo_atualizado_original}': {e}. Nomes serão buscados online para todos.")


    # 3. Iterar sobre os códigos de referência e buscar nomes
    nomes_encontrados_online = 0
    nomes_usados_preexistentes = 0
    erros_busca = 0

    for i, codigo in enumerate(todos_codigos_referencia):
        print(f"Processando {i+1}/{len(todos_codigos_referencia)}: {codigo}")
        nome_empresa = None

        # Verificar se já temos o nome na planilha "Atualizado"
        if codigo in nomes_pre_existentes:
            nome_empresa = nomes_pre_existentes[codigo]
            logging.info(f"Nome para {codigo} encontrado na planilha pré-existente: '{nome_empresa}'")
            nomes_usados_preexistentes += 1
        
        # Se não encontrou ou nome é inválido, buscar online
        if not nome_empresa or not str(nome_empresa).strip():
            nome_empresa_online = buscar_nome_empresa_site(codigo)
            if nome_empresa_online:
                nome_empresa = nome_empresa_online
                nomes_encontrados_online +=1
            else:
                logging.warning(f"Não foi possível encontrar nome online para {codigo}.")
                erros_busca +=1
            time.sleep(1.2) # Pausa para ser gentil com os servidores

        dados_finais.append({
            COLUNA_CODIGO_FINAL: codigo,
            COLUNA_NOME_FINAL: nome_empresa if nome_empresa else pd.NA # Usar pd.NA para Excel
        })

    # 4. Criar DataFrame e salvar
    df_final = pd.DataFrame(dados_finais)
    
    try:
        df_final.to_excel(ARQUIVO_FINAL_COMPLETO, index=False)
        print(f"\nPlanilha final gerada com sucesso em: {ARQUIVO_FINAL_COMPLETO}")
        print("--- Resumo da Geração ---")
        print(f"Total de códigos processados: {len(todos_codigos_referencia)}")
        print(f"Nomes reutilizados da planilha pré-existente: {nomes_usados_preexistentes}")
        print(f"Nomes encontrados online nesta execução: {nomes_encontrados_online}")
        print(f"Códigos para os quais não foi possível obter nome: {erros_busca}")
        print(f"Total de empresas na planilha final: {len(df_final)}")

    except Exception as e:
        print(f"ERRO ao salvar a planilha final '{ARQUIVO_FINAL_COMPLETO}': {e}")

if __name__ == "__main__":
    main()

Iniciando processo de coleta e preenchimento de nomes de empresas...
Carregados e desmembrados 385 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx' da coluna 'cod_negociacao'.
Carregados e desmembrados 382 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx' da coluna 'cod_negociacao'.
Total de 400 códigos de negociação únicos para processar.
Carregada planilha 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx' para verificar nomes pré-existentes.
Encontrados 65 nomes na planilha pré-existente.
Processando 1/400: AALR3


2025-05-20 15:30:42,806 - WARNING - Nome da empresa NÃO encontrado para AALR3 após tentar todos os sites/URLs.
2025-05-20 15:30:42,806 - WARNING - Não foi possível encontrar nome online para AALR3.
2025-05-20 15:30:44,008 - INFO - Nome para ABCB4 encontrado na planilha pré-existente: 'ABC BRASIL'
2025-05-20 15:30:44,008 - INFO - Nome para ABEV3 encontrado na planilha pré-existente: 'AMBEV S/A'


Processando 2/400: ABCB4
Processando 3/400: ABEV3
Processando 4/400: AERI3


2025-05-20 15:30:45,902 - WARNING - Nome da empresa NÃO encontrado para AERI3 após tentar todos os sites/URLs.
2025-05-20 15:30:45,902 - WARNING - Não foi possível encontrar nome online para AERI3.


Processando 5/400: AESB3


...

Processando 399/400: YDUQ3


2025-05-20 15:47:48,474 - WARNING - Nome da empresa NÃO encontrado para YDUQ3 após tentar todos os sites/URLs.
2025-05-20 15:47:48,478 - WARNING - Não foi possível encontrar nome online para YDUQ3.


Processando 400/400: ZAMP3


2025-05-20 15:47:51,521 - WARNING - Nome da empresa NÃO encontrado para ZAMP3 após tentar todos os sites/URLs.
2025-05-20 15:47:51,521 - WARNING - Não foi possível encontrar nome online para ZAMP3.



Planilha final gerada com sucesso em: C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL.xlsx
--- Resumo da Geração ---
Total de códigos processados: 400
Nomes reutilizados da planilha pré-existente: 64
Nomes encontrados online nesta execução: 0
Códigos para os quais não foi possível obter nome: 336
Total de empresas na planilha final: 400


In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import time
import re # Para expressões regulares (limpeza de nome)

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuração dos Nomes dos Arquivos e Colunas ---
ARQUIVO_ANTERIOR = r"C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx"
COLUNA_CODIGO_ANTERIOR = "cod_negociacao"

ARQUIVO_ATUAL = r"C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx"
COLUNA_CODIGO_ATUAL = "cod_negociacao"

# Arquivo NOVO a ser GERADO
ARQUIVO_FINAL_COMPLETO = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL.xlsx"
COLUNA_CODIGO_FINAL = "Codigo Negociacao"
COLUNA_NOME_FINAL = "Empresa Capital Aberto"


def carregar_e_desmembrar_codigos(filepath, column_name):
    """Carrega códigos, desmembra se estiverem agrupados, e normaliza."""
    codigos_finais = set()
    try:
        df = pd.read_excel(filepath, usecols=[column_name])
        raw_codigos = df[column_name].dropna().astype(str).str.strip().str.upper()
        
        for item_lista_codigos in raw_codigos:
            # Separa por vírgula, ponto e vírgula ou espaço, e remove espaços extras de cada parte
            partes = re.split(r'[,;\s]+', item_lista_codigos)
            for codigo_individual in partes:
                codigo_limpo = codigo_individual.strip()
                if codigo_limpo: # Adiciona apenas se não for uma string vazia
                    codigos_finais.add(codigo_limpo)
        
        logging.info(f"Carregados e desmembrados {len(codigos_finais)} códigos únicos de '{filepath}' da coluna '{column_name}'.")
        return codigos_finais
    except FileNotFoundError:
        logging.error(f"ERRO: Arquivo não encontrado: {filepath}")
        return set()
    except ValueError as ve:
        logging.error(f"ERRO: Coluna '{column_name}' não encontrada em '{filepath}' ou outro erro de valor: {ve}")
        try:
            df_full = pd.read_excel(filepath)
            logging.info(f"Colunas disponíveis em '{filepath}': {list(df_full.columns)}")
        except Exception as e_full:
            logging.error(f"Não foi possível ler as colunas de '{filepath}': {e_full}")
        return set()
    except Exception as e:
        logging.error(f"ERRO ao ler o arquivo '{filepath}': {e}")
        return set()

def buscar_nome_empresa_site(codigo_negociacao):
    """Busca o nome da empresa no site investsite.com.br ou infomoney."""
    if not codigo_negociacao or pd.isna(codigo_negociacao):
        logging.warning("buscar_nome_empresa_site: Código de negociação inválido ou ausente.")
        return None

    codigo_fmt = str(codigo_negociacao).strip().upper()
    urls_e_parsers = []

    # --- InvestSite ---
    investsite_urls = [
        f"https://www.investsite.com.br/principais_acoes.php?cod_negociacao={codigo_fmt}",
        f"https://www.investsite.com.br/fundos_imobiliarios_detalhes.php?cod_negociacao={codigo_fmt}"
    ]
    for url in investsite_urls:
        urls_e_parsers.append({
            "url": url,
            "parser_func": parse_investsite_nome,
            "site_name": "InvestSite"
        })
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }

    for item in urls_e_parsers:
        url = item["url"]
        parser_func = item["parser_func"]
        site_name = item["site_name"]

        logging.info(f"--- Tentando {site_name} para {codigo_fmt} ---")
        logging.info(f"URL: {url}")
        try:
            response = requests.get(url, headers=headers, timeout=20) 
            logging.info(f"Status Code da Resposta: {response.status_code}")

            # Salvar o HTML para análise (DESCOMENTE E AJUSTE O CODIGO_FMT PARA TESTAR UM ESPECÍFICO)
            # if codigo_fmt == "AALR3": # Exemplo, substitua pelo código que quer depurar
            #    filename_html = f"debug_html_{codigo_fmt}_{site_name.replace(' ', '_')}.html"
            #    with open(filename_html, "w", encoding="utf-8") as f_html:
            #        f_html.write(response.text)
            #    logging.info(f"HTML salvo para {codigo_fmt} em {filename_html}")


            if response.status_code == 404:
                 logging.warning(f"Página não encontrada (404) em {site_name} para {codigo_fmt} na URL: {url}")
                 continue 
            response.raise_for_status() 
            
            soup = BeautifulSoup(response.content, 'lxml')
            nome_empresa = parser_func(soup, codigo_fmt)

            if nome_empresa:
                logging.info(f"SUCESSO: Nome da empresa encontrado para {codigo_fmt} em {site_name}: {nome_empresa}")
                return nome_empresa
            else:
                logging.warning(f"FALHA no PARSE: Nome NÃO extraído por parser_func para {codigo_fmt} em {site_name} (URL: {url})")
        
        except requests.exceptions.HTTPError as e:
            logging.error(f"Erro HTTP ao buscar em {site_name} para {codigo_fmt} (URL: {url}): {e}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Erro de requisição ao buscar em {site_name} para {codigo_fmt} (URL: {url}): {e}")
        except Exception as e:
            logging.error(f"Erro inesperado ao processar {site_name} para {codigo_fmt} (URL: {url}): {e}")
        
        logging.info(f"--- Fim da tentativa {site_name} para {codigo_fmt} ---")
        time.sleep(0.5) 

    logging.warning(f"FINAL: Nome da empresa NÃO encontrado para {codigo_fmt} após tentar todas as fontes.")
    return None

def parse_investsite_nome(soup, codigo_negociacao):
    """Extrai o nome da empresa da página do InvestSite."""
    logging.debug(f"parse_investsite_nome: Iniciando parse para {codigo_negociacao}")
    h1_tag = soup.find('h1')
    
    if not h1_tag:
        logging.warning(f"parse_investsite_nome: Tag <h1> NÃO encontrada no HTML para {codigo_negociacao}.")
        error_message_div = soup.find('div', class_='alert alert-danger') 
        if error_message_div:
            logging.warning(f"parse_investsite_nome: Encontrada possível mensagem de erro na página: {error_message_div.get_text(strip=True)}")
        return None

    full_text_h1 = h1_tag.get_text(separator=" ", strip=True)
    logging.info(f"parse_investsite_nome: Texto encontrado em <h1>: '{full_text_h1}' para {codigo_negociacao}")
    nome_empresa = full_text_h1
    
    small_tag = h1_tag.find('small') 
    if small_tag:
        small_tag_text = small_tag.get_text(strip=True)
        logging.debug(f"parse_investsite_nome: Texto da tag <small>: '{small_tag_text}'")
        h1_copy = BeautifulSoup(str(h1_tag), 'lxml')
        if h1_copy.h1 and h1_copy.h1.small:
             h1_copy.h1.small.decompose() 
        nome_empresa = h1_copy.h1.get_text(separator=" ", strip=True)
        logging.info(f"parse_investsite_nome: Nome após remover <small>: '{nome_empresa}'")
    else:
        logging.debug(f"parse_investsite_nome: Tag <small> não encontrada dentro do <h1>.")
    
    nome_empresa = re.sub(r'\b' + re.escape(codigo_negociacao) + r'\b', '', nome_empresa, flags=re.IGNORECASE).strip()
    logging.debug(f"parse_investsite_nome: Nome após remover código '{codigo_negociacao}': '{nome_empresa}'")
    
    sufixos_comuns = ["ON", "PN", "PNA", "PNB", "UNT", "EDJ", "ED", "EJ", "CI", "ERJ"] 
    
    for sufixo in sufixos_comuns:
        pattern = r"(\s*" + re.escape(sufixo) + r")$" 
        if nome_empresa and re.search(pattern, nome_empresa, flags=re.IGNORECASE):
            nome_empresa_antes_sufixo = nome_empresa
            nome_empresa = re.sub(pattern, "", nome_empresa, flags=re.IGNORECASE).strip()
            logging.debug(f"parse_investsite_nome: Removido sufixo '{sufixo}'. Antes: '{nome_empresa_antes_sufixo}', Depois: '{nome_empresa}'")

    nome_empresa = re.sub(r'\s+', ' ', nome_empresa).strip()
    logging.debug(f"parse_investsite_nome: Nome final após limpeza de espaços: '{nome_empresa}'")

    if nome_empresa:
        logging.info(f"parse_investsite_nome: Nome extraído com sucesso: '{nome_empresa}' para {codigo_negociacao}")
        return nome_empresa
    else:
        logging.warning(f"parse_investsite_nome: Nome resultou VAZIO após todas as limpezas para {codigo_negociacao}. Texto H1 original: '{full_text_h1}'")
        return None

# def parse_infomoney_nome(soup, codigo_negociacao):
#     """Extrai o nome da empresa da página do InfoMoney (PRECISA SER IMPLEMENTADO CORRETAMENTE)."""
#     # ... (código comentado do InfoMoney, como antes) ...
#     return None


def main():
    print("Iniciando processo de coleta e preenchimento de nomes de empresas...")

    # 1. Carregar e unificar todos os códigos de referência
    codigos_ref_anterior = carregar_e_desmembrar_codigos(ARQUIVO_ANTERIOR, COLUNA_CODIGO_ANTERIOR)
    codigos_ref_atual = carregar_e_desmembrar_codigos(ARQUIVO_ATUAL, COLUNA_CODIGO_ATUAL)
    
    todos_codigos_referencia = sorted(list(codigos_ref_anterior.union(codigos_ref_atual)))
    
    if not todos_codigos_referencia:
        print("Nenhum código de referência encontrado. Abortando.")
        return
    
    print(f"Total de {len(todos_codigos_referencia)} códigos de negociação únicos para processar.")

    # 2. Preparar dados para a nova planilha
    dados_finais = [] 

    nomes_pre_existentes = {}
    arquivo_atualizado_original = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx"
    col_cod_atualizado = "Codigo Negociacao" 
    col_nome_atualizado = "Empresa Capital Aberto"

    try:
        df_atualizado_orig = pd.read_excel(arquivo_atualizado_original)
        logging.info(f"Carregada planilha '{arquivo_atualizado_original}' para verificar nomes pré-existentes.")
        for _, row in df_atualizado_orig.iterrows():
            cod_orig = row.get(col_cod_atualizado)
            nome_orig = row.get(col_nome_atualizado)

            if pd.notna(cod_orig) and pd.notna(nome_orig) and str(nome_orig).strip():
                cod_orig_str = str(cod_orig).strip().upper()
                partes_cod_orig = re.split(r'[,;\s]+', cod_orig_str)
                for c in partes_cod_orig:
                    if c and c not in nomes_pre_existentes: 
                         nomes_pre_existentes[c] = str(nome_orig).strip()
        logging.info(f"Encontrados {len(nomes_pre_existentes)} nomes na planilha pré-existente.")

    except FileNotFoundError:
        logging.info(f"Arquivo '{arquivo_atualizado_original}' não encontrado. Nomes serão buscados online para todos.")
    except Exception as e:
        logging.error(f"Erro ao ler '{arquivo_atualizado_original}': {e}. Nomes serão buscados online para todos.")


    # 3. Iterar sobre os códigos de referência e buscar nomes
    nomes_encontrados_online = 0
    nomes_usados_preexistentes = 0
    erros_busca = 0

    for i, codigo in enumerate(todos_codigos_referencia):
        logging.info(f"--- Processando {i+1}/{len(todos_codigos_referencia)}: {codigo} ---")
        nome_empresa = None

        if codigo in nomes_pre_existentes:
            nome_empresa = nomes_pre_existentes[codigo]
            logging.info(f"Nome para {codigo} encontrado na planilha pré-existente: '{nome_empresa}'")
            nomes_usados_preexistentes += 1
        
        if not nome_empresa or not str(nome_empresa).strip():
            logging.info(f"Nome para {codigo} não encontrado/inválido na pré-existente. Buscando online...")
            nome_empresa_online = buscar_nome_empresa_site(codigo)
            if nome_empresa_online:
                nome_empresa = nome_empresa_online
                nomes_encontrados_online +=1
            else:
                logging.warning(f"Não foi possível encontrar nome online para {codigo}.")
                erros_busca +=1
            time.sleep(1.5) # Aumentei a pausa um pouco mais
        else:
            # Se usou nome pré-existente e ele era válido, não precisa de pausa.
            pass


        dados_finais.append({
            COLUNA_CODIGO_FINAL: codigo,
            COLUNA_NOME_FINAL: nome_empresa if nome_empresa else pd.NA 
        })

    # 4. Criar DataFrame e salvar
    df_final = pd.DataFrame(dados_finais)
    
    try:
        df_final.to_excel(ARQUIVO_FINAL_COMPLETO, index=False)
        print(f"\nPlanilha final gerada com sucesso em: {ARQUIVO_FINAL_COMPLETO}")
        print("--- Resumo da Geração ---")
        print(f"Total de códigos processados: {len(todos_codigos_referencia)}")
        print(f"Nomes reutilizados da planilha pré-existente: {nomes_usados_preexistentes}")
        print(f"Nomes encontrados online nesta execução: {nomes_encontrados_online}")
        print(f"Códigos para os quais não foi possível obter nome: {erros_busca}")
        print(f"Total de empresas na planilha final: {len(df_final)}")

    except Exception as e:
        print(f"ERRO ao salvar a planilha final '{ARQUIVO_FINAL_COMPLETO}': {e}")

if __name__ == "__main__":
    main()

2025-05-20 15:53:11,357 - INFO - Carregados e desmembrados 385 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx' da coluna 'cod_negociacao'.
2025-05-20 15:53:11,461 - INFO - Carregados e desmembrados 382 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx' da coluna 'cod_negociacao'.


Iniciando processo de coleta e preenchimento de nomes de empresas...


2025-05-20 15:53:11,477 - INFO - Carregada planilha 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx' para verificar nomes pré-existentes.
2025-05-20 15:53:11,481 - INFO - Encontrados 65 nomes na planilha pré-existente.
2025-05-20 15:53:11,482 - INFO - --- Processando 1/400: AALR3 ---
2025-05-20 15:53:11,482 - INFO - Nome para AALR3 não encontrado/inválido na pré-existente. Buscando online...
2025-05-20 15:53:11,483 - INFO - --- Tentando InvestSite para AALR3 ---
2025-05-20 15:53:11,483 - INFO - URL: https://www.investsite.com.br/principais_acoes.php?cod_negociacao=AALR3


Total de 400 códigos de negociação únicos para processar.


2025-05-20 15:53:12,455 - INFO - Status Code da Resposta: 404
2025-05-20 15:53:12,455 - WARNING - Página não encontrada (404) em InvestSite para AALR3 na URL: https://www.investsite.com.br/principais_acoes.php?cod_negociacao=AALR3
2025-05-20 15:53:12,456 - INFO - --- Tentando InvestSite para AALR3 ---
2025-05-20 15:53:12,456 - INFO - URL: https://www.investsite.com.br/fundos_imobiliarios_detalhes.php?cod_negociacao=AALR3
2025-05-20 15:53:13,418 - INFO - Status Code da Resposta: 404
2025-05-20 15:53:13,418 - WARNING - Página não encontrada (404) em InvestSite para AALR3 na URL: https://www.investsite.com.br/fundos_imobiliarios_detalhes.php?cod_negociacao=AALR3
2025-05-20 15:53:13,420 - WARNING - FINAL: Nome da empresa NÃO encontrado para AALR3 após tentar todas as fontes.
2025-05-20 15:53:13,422 - WARNING - Não foi possível encontrar nome online para AALR3.
2025-05-20 15:53:14,922 - INFO - --- Processando 2/400: ABCB4 ---
2025-05-20 15:53:14,922 - INFO - Nome para ABCB4 encontrado na pl

KeyboardInterrupt: 

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import time
import re

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuração dos Nomes dos Arquivos e Colunas ---
ARQUIVO_ANTERIOR = r"C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx"
COLUNA_CODIGO_ANTERIOR = "cod_negociacao"

ARQUIVO_ATUAL = r"C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx"
COLUNA_CODIGO_ATUAL = "cod_negociacao"

# Arquivo NOVO a ser GERADO
ARQUIVO_FINAL_COMPLETO = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL_COM_SETORES.xlsx"
COLUNA_CODIGO_FINAL = "Codigo Negociacao"
COLUNA_NOME_FINAL = "Empresa Capital Aberto"
COLUNA_SETOR1 = "Setor_Atuacao"  # Coluna C da sua descrição original
COLUNA_SETOR2 = "Setor_Atuacao2" # Coluna E da sua descrição original (assumindo D é igual C)
COLUNA_SETOR3 = "Setor_Atuacao3" # Coluna F da sua descrição original

# Nota: Se "Setor_Atuacao" (coluna D) for diferente de "Setor_Atuação" (coluna C),
# você precisará adicionar outra variável COLUNA_SETOR_D e mapear os dados para ela.
# Por simplicidade, estou assumindo que "Setor_Atuação" (C) é o principal e
# os outros (E, F) são os subsequentes.


def carregar_e_desmembrar_codigos(filepath, column_name):
    """Carrega códigos, desmembra se estiverem agrupados, e normaliza."""
    codigos_finais = set()
    try:
        df = pd.read_excel(filepath, usecols=[column_name])
        raw_codigos = df[column_name].dropna().astype(str).str.strip().str.upper()
        
        for item_lista_codigos in raw_codigos:
            partes = re.split(r'[,;\s]+', item_lista_codigos)
            for codigo_individual in partes:
                codigo_limpo = codigo_individual.strip()
                if codigo_limpo:
                    codigos_finais.add(codigo_limpo)
        
        logging.info(f"Carregados e desmembrados {len(codigos_finais)} códigos únicos de '{filepath}' da coluna '{column_name}'.")
        return codigos_finais
    except FileNotFoundError:
        logging.error(f"ERRO: Arquivo não encontrado: {filepath}")
        return set()
    except ValueError as ve:
        logging.error(f"ERRO: Coluna '{column_name}' não encontrada em '{filepath}' ou outro erro de valor: {ve}")
        try:
            df_full = pd.read_excel(filepath)
            logging.info(f"Colunas disponíveis em '{filepath}': {list(df_full.columns)}")
        except Exception as e_full:
            logging.error(f"Não foi possível ler as colunas de '{filepath}': {e_full}")
        return set()
    except Exception as e:
        logging.error(f"ERRO ao ler o arquivo '{filepath}': {e}")
        return set()

def find_value_for_label_in_table(soup, label_text_options):
    """
    Procura por um label em uma tabela e retorna o valor da célula adjacente.
    label_text_options é uma lista de possíveis textos para o label (case-insensitive).
    """
    if not isinstance(label_text_options, list):
        label_text_options = [label_text_options]
    
    label_text_options_lower = [opt.lower() for opt in label_text_options]

    # A estrutura no InvestSite parece ser <td>LABEL</td><td>VALOR</td>
    # dentro de um <tbody> de uma <table>.
    tables = soup.find_all('table')
    for table in tables:
        for tr in table.find_all('tr'):
            cells = tr.find_all('td')
            if len(cells) >= 2:
                label_cell_text = cells[0].get_text(strip=True).lower()
                
                for opt_lower in label_text_options_lower:
                    if opt_lower in label_cell_text: # Usar 'in' para correspondência parcial se necessário
                        value_cell = cells[1]
                        # O valor pode estar dentro de um link <a> ou direto
                        link_tag = value_cell.find('a')
                        if link_tag:
                            return link_tag.get_text(strip=True)
                        else:
                            return value_cell.get_text(strip=True)
    logging.debug(f"Label(s) '{label_text_options}' não encontrado(s) na estrutura de tabela esperada.")
    return None


def buscar_dados_empresa_investsite(codigo_negociacao):
    """
    Busca Razão Social, Setor, Subsetor e Segmento da página de indicadores principais.
    Retorna um dicionário com os dados encontrados.
    """
    if not codigo_negociacao or pd.isna(codigo_negociacao):
        logging.warning("buscar_dados_empresa_investsite: Código de negociação inválido ou ausente.")
        return None

    codigo_fmt = str(codigo_negociacao).strip().upper()
    url = f"https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={codigo_fmt}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7', # Priorizar pt-BR
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }

    logging.info(f"--- Buscando dados completos para {codigo_fmt} ---")
    logging.info(f"URL: {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=20) 
        logging.info(f"Status Code da Resposta: {response.status_code}")

        # Salvar o HTML para análise (DESCOMENTE E AJUSTE O CODIGO_FMT PARA TESTAR UM ESPECÍFICO)
        # if codigo_fmt == "BMGB4": # Exemplo
        #    filename_html = f"debug_html_indicadores_{codigo_fmt}.html"
        #    with open(filename_html, "w", encoding="utf-8") as f_html:
        #        f_html.write(response.text)
        #    logging.info(f"HTML salvo para {codigo_fmt} em {filename_html}")

        if response.status_code == 404:
             logging.warning(f"Página não encontrada (404) para {codigo_fmt} na URL: {url}")
             return None
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.content, 'lxml')
        
        dados_empresa = {}

        # Extrair Razão Social
        razao_social = find_value_for_label_in_table(soup, ["Razão Social"])
        if razao_social:
            dados_empresa['nome'] = razao_social
            logging.info(f"Razão Social para {codigo_fmt}: '{razao_social}'")
        else:
            logging.warning(f"Razão Social NÃO encontrada para {codigo_fmt}.")
            # Fallback: tentar pegar do H1 (pode ser nome fantasia)
            h1_tag = soup.find('h1')
            if h1_tag:
                h1_text = h1_tag.get_text(separator=" ", strip=True)
                if h1_text:
                     # Tentar limpar o código do H1
                    nome_fantasia = re.sub(r'\(' + re.escape(codigo_fmt) + r'\)', '', h1_text, flags=re.IGNORECASE).strip()
                    nome_fantasia = re.sub(r'\b' + re.escape(codigo_fmt) + r'\b', '', nome_fantasia, flags=re.IGNORECASE).strip()
                    if nome_fantasia:
                        dados_empresa['nome'] = nome_fantasia # Usa nome fantasia se razao social falhar
                        logging.info(f"Nome (H1 fallback) para {codigo_fmt}: '{nome_fantasia}'")


        # Extrair Setor, Subsetor, Segmento
        dados_empresa['setor1'] = find_value_for_label_in_table(soup, ["Setor"])
        dados_empresa['setor2'] = find_value_for_label_in_table(soup, ["Subsetor"])
        dados_empresa['setor3'] = find_value_for_label_in_table(soup, ["Segmento"])

        logging.info(f"Setor para {codigo_fmt}: '{dados_empresa.get('setor1')}'")
        logging.info(f"Subsetor para {codigo_fmt}: '{dados_empresa.get('setor2')}'")
        logging.info(f"Segmento para {codigo_fmt}: '{dados_empresa.get('setor3')}'")

        # Retorna o dicionário apenas se pelo menos o nome foi encontrado
        if dados_empresa.get('nome'):
            return dados_empresa
        else:
            logging.warning(f"Nenhum nome (Razão Social ou H1) encontrado para {codigo_fmt} na página de indicadores.")
            return None
            
    except requests.exceptions.HTTPError as e:
        logging.error(f"Erro HTTP ao buscar dados completos para {codigo_fmt} (URL: {url}): {e}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro de requisição ao buscar dados completos para {codigo_fmt} (URL: {url}): {e}")
    except Exception as e:
        logging.error(f"Erro inesperado ao processar dados completos para {codigo_fmt} (URL: {url}): {e}")
    
    logging.warning(f"FINAL: Dados completos NÃO encontrados para {codigo_fmt}.")
    return None


def main():
    print("Iniciando processo de coleta e preenchimento de informações de empresas...")

    codigos_ref_anterior = carregar_e_desmembrar_codigos(ARQUIVO_ANTERIOR, COLUNA_CODIGO_ANTERIOR)
    codigos_ref_atual = carregar_e_desmembrar_codigos(ARQUIVO_ATUAL, COLUNA_CODIGO_ATUAL)
    
    todos_codigos_referencia = sorted(list(codigos_ref_anterior.union(codigos_ref_atual)))
    
    if not todos_codigos_referencia:
        print("Nenhum código de referência encontrado. Abortando.")
        return
    
    print(f"Total de {len(todos_codigos_referencia)} códigos de negociação únicos para processar.")

    dados_finais_lista = [] 
    nomes_pre_existentes_map = {} # Usar um mapa para nomes e setores pré-existentes

    # Tentar carregar a planilha já existente para reutilizar dados se possível
    # Isso pode ser o arquivo de uma execução anterior deste mesmo script ou o "Atualizado"
    arquivo_cache_dados = ARQUIVO_FINAL_COMPLETO # Tenta ler o próprio arquivo de saída de execuções anteriores
    # Ou: arquivo_cache_dados = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_Nomes_Codigos_Atualizado.xlsx"

    try:
        df_cache = pd.read_excel(arquivo_cache_dados)
        logging.info(f"Carregada planilha cache '{arquivo_cache_dados}' para verificar dados pré-existentes.")
        for _, row in df_cache.iterrows():
            cod_orig = row.get(COLUNA_CODIGO_FINAL) # Usar colunas do arquivo final
            
            if pd.notna(cod_orig):
                cod_orig_str = str(cod_orig).strip().upper()
                # Não precisa desmembrar aqui, pois o arquivo final deve ter códigos únicos por linha
                if cod_orig_str not in nomes_pre_existentes_map:
                    nomes_pre_existentes_map[cod_orig_str] = {
                        'nome': str(row.get(COLUNA_NOME_FINAL, "")).strip(),
                        'setor1': str(row.get(COLUNA_SETOR1, "")).strip(),
                        'setor2': str(row.get(COLUNA_SETOR2, "")).strip(),
                        'setor3': str(row.get(COLUNA_SETOR3, "")).strip(),
                    }
        logging.info(f"Encontrados dados para {len(nomes_pre_existentes_map)} códigos na planilha cache.")
    except FileNotFoundError:
        logging.info(f"Arquivo cache '{arquivo_cache_dados}' não encontrado. Todos os dados serão buscados online se necessário.")
    except Exception as e:
        logging.error(f"Erro ao ler arquivo cache '{arquivo_cache_dados}': {e}. Dados serão buscados online.")

    dados_coletados_online = 0
    dados_usados_cache = 0
    erros_busca_total = 0

    for i, codigo in enumerate(todos_codigos_referencia):
        logging.info(f"--- Processando {i+1}/{len(todos_codigos_referencia)}: {codigo} ---")
        
        dados_empresa_final = {
            COLUNA_CODIGO_FINAL: codigo,
            COLUNA_NOME_FINAL: pd.NA,
            COLUNA_SETOR1: pd.NA,
            COLUNA_SETOR2: pd.NA,
            COLUNA_SETOR3: pd.NA
        }

        # Tentar usar dados do cache primeiro
        if codigo in nomes_pre_existentes_map:
            cached_data = nomes_pre_existentes_map[codigo]
            # Usar dados do cache apenas se o nome for válido
            if cached_data.get('nome'):
                dados_empresa_final[COLUNA_NOME_FINAL] = cached_data['nome']
                dados_empresa_final[COLUNA_SETOR1] = cached_data.get('setor1') or pd.NA
                dados_empresa_final[COLUNA_SETOR2] = cached_data.get('setor2') or pd.NA
                dados_empresa_final[COLUNA_SETOR3] = cached_data.get('setor3') or pd.NA
                logging.info(f"Dados para {codigo} encontrados no cache: Nome='{cached_data['nome']}', Setor1='{cached_data.get('setor1')}'")
                dados_usados_cache += 1
            else: # Nome no cache era inválido, precisa buscar online
                 logging.info(f"Nome para {codigo} inválido no cache. Buscando online...")
        else: # Não está no cache, precisa buscar online
            logging.info(f"Dados para {codigo} não encontrados no cache. Buscando online...")

        # Se o nome ainda não foi preenchido (seja por não estar no cache ou por ser inválido no cache)
        if pd.isna(dados_empresa_final[COLUNA_NOME_FINAL]):
            dados_online = buscar_dados_empresa_investsite(codigo)
            if dados_online:
                dados_empresa_final[COLUNA_NOME_FINAL] = dados_online.get('nome', pd.NA)
                dados_empresa_final[COLUNA_SETOR1] = dados_online.get('setor1', pd.NA)
                dados_empresa_final[COLUNA_SETOR2] = dados_online.get('setor2', pd.NA)
                dados_empresa_final[COLUNA_SETOR3] = dados_online.get('setor3', pd.NA)
                dados_coletados_online += 1
            else:
                logging.warning(f"Não foi possível obter dados online para {codigo}.")
                erros_busca_total += 1
            time.sleep(1.5) # Pausa sempre que uma busca online é tentada
        
        dados_finais_lista.append(dados_empresa_final)

    df_final = pd.DataFrame(dados_finais_lista)
    
    try:
        df_final.to_excel(ARQUIVO_FINAL_COMPLETO, index=False)
        print(f"\nPlanilha final gerada com sucesso em: {ARQUIVO_FINAL_COMPLETO}")
        print("--- Resumo da Geração ---")
        print(f"Total de códigos processados: {len(todos_codigos_referencia)}")
        print(f"Dados reutilizados do cache: {dados_usados_cache}")
        print(f"Dados coletados online nesta execução: {dados_coletados_online}")
        print(f"Códigos para os quais não foi possível obter dados completos: {erros_busca_total}")
        print(f"Total de registros na planilha final: {len(df_final)}")

    except Exception as e:
        print(f"ERRO ao salvar a planilha final '{ARQUIVO_FINAL_COMPLETO}': {e}")

if __name__ == "__main__":
    main()

2025-05-20 16:05:16,010 - INFO - Carregados e desmembrados 385 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx' da coluna 'cod_negociacao'.
2025-05-20 16:05:16,105 - INFO - Carregados e desmembrados 382 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx' da coluna 'cod_negociacao'.


Iniciando processo de coleta e preenchimento de informações de empresas...


2025-05-20 16:05:16,107 - INFO - Arquivo cache 'C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL_COM_SETORES.xlsx' não encontrado. Todos os dados serão buscados online se necessário.
2025-05-20 16:05:16,107 - INFO - --- Processando 1/400: AALR3 ---
2025-05-20 16:05:16,108 - INFO - Dados para AALR3 não encontrados no cache. Buscando online...
2025-05-20 16:05:16,108 - INFO - --- Buscando dados completos para AALR3 ---
2025-05-20 16:05:16,109 - INFO - URL: https://www.investsite.com.br/principais_indicadores.php?cod_negociacao=AALR3


Total de 400 códigos de negociação únicos para processar.


2025-05-20 16:05:17,221 - INFO - Status Code da Resposta: 200
2025-05-20 16:05:17,257 - INFO - Razão Social para AALR3: 'ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.'
2025-05-20 16:05:17,260 - INFO - Setor para AALR3: 'Saúde'
2025-05-20 16:05:17,260 - INFO - Subsetor para AALR3: 'Serv.Méd.Hospit..Análises e Diagnósticos'
2025-05-20 16:05:17,260 - INFO - Segmento para AALR3: 'Novo Mercado'
2025-05-20 16:05:18,763 - INFO - --- Processando 2/400: ABCB4 ---
2025-05-20 16:05:18,763 - INFO - Dados para ABCB4 não encontrados no cache. Buscando online...
2025-05-20 16:05:18,764 - INFO - --- Buscando dados completos para ABCB4 ---
2025-05-20 16:05:18,765 - INFO - URL: https://www.investsite.com.br/principais_indicadores.php?cod_negociacao=ABCB4
2025-05-20 16:05:19,868 - INFO - Status Code da Resposta: 200
2025-05-20 16:05:19,903 - INFO - Razão Social para ABCB4: 'BCO ABC BRASIL S.A.'
2025-05-20 16:05:19,907 - INFO - Setor para ABCB4: 'Financeiro'
2025-05-20 16:05:19,907 - INFO - Subsetor para ABCB4: 'In


Planilha final gerada com sucesso em: C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL_COM_SETORES.xlsx
--- Resumo da Geração ---
Total de códigos processados: 400
Dados reutilizados do cache: 0
Dados coletados online nesta execução: 360
Códigos para os quais não foi possível obter dados completos: 40
Total de registros na planilha final: 400


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import logging
import time
import re
import os # Para limpar o cache

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configuração dos Nomes dos Arquivos e Colunas ---
ARQUIVO_ANTERIOR = r"C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx"
COLUNA_CODIGO_ANTERIOR = "cod_negociacao"

ARQUIVO_ATUAL = r"C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx"
COLUNA_CODIGO_ATUAL = "cod_negociacao"

# Arquivo NOVO a ser GERADO
ARQUIVO_FINAL_COMPLETO = r"C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL_CORRETO.xlsx"
COLUNA_CODIGO_FINAL = "Codigo Negociacao"
COLUNA_NOME_FINAL = "Empresa Capital Aberto"
COLUNA_SETOR_C = "Setor_Atuacao"    # Coluna C (Label "Setor" do site)
COLUNA_SETOR_D = "Setor_Atuacao2"   # Coluna D (Label "Subsetor" do site)
COLUNA_SETOR_E = "Setor_Atuacao3"   # Coluna E (Label "Segmento" do site - o que acompanha Setor/Subsetor)


def carregar_e_desmembrar_codigos(filepath, column_name):
    # ... (função sem alterações) ...
    codigos_finais = set()
    try:
        df = pd.read_excel(filepath, usecols=[column_name])
        raw_codigos = df[column_name].dropna().astype(str).str.strip().str.upper()
        
        for item_lista_codigos in raw_codigos:
            partes = re.split(r'[,;\s]+', item_lista_codigos)
            for codigo_individual in partes:
                codigo_limpo = codigo_individual.strip()
                if codigo_limpo:
                    codigos_finais.add(codigo_limpo)
        
        logging.info(f"Carregados e desmembrados {len(codigos_finais)} códigos únicos de '{filepath}' da coluna '{column_name}'.")
        return codigos_finais
    except FileNotFoundError:
        logging.error(f"ERRO: Arquivo não encontrado: {filepath}")
        return set()
    except ValueError as ve:
        logging.error(f"ERRO: Coluna '{column_name}' não encontrada em '{filepath}' ou outro erro de valor: {ve}")
        try:
            df_full = pd.read_excel(filepath)
            logging.info(f"Colunas disponíveis em '{filepath}': {list(df_full.columns)}")
        except Exception as e_full:
            logging.error(f"Não foi possível ler as colunas de '{filepath}': {e_full}")
        return set()
    except Exception as e:
        logging.error(f"ERRO ao ler o arquivo '{filepath}': {e}")
        return set()


def extract_data_from_specific_table(soup, labels_map):
    """
    Extrai dados de uma tabela específica que contém os "Dados Básicos da Empresa".
    labels_map: um dicionário como {'Razão Social': 'nome_desejado_no_output', 'Setor': 'setor_desejado'}
    """
    dados_encontrados = {}
    # Tentar encontrar a tabela pela legenda/cabeçalho, se houver.
    # Como alternativa, inspecionar o HTML e ver se a tabela tem um ID ou classe específica.
    # Por enquanto, vamos assumir que é uma das primeiras tabelas com a estrutura <td>Label</td><td>Value</td>
    
    # Heurística: A tabela com "Setor", "Subsetor", "Segmento" geralmente é a que tem "Dados Básicos da Empresa"
    # Vamos procurar por uma tabela que contenha o label "Empresa" ou "Razão Social" como um bom indicador.
    
    potential_tables = []
    all_tables = soup.find_all('table')

    # Primeiro, tentar identificar a tabela de "Dados Básicos da Empresa"
    # Esta tabela geralmente contém "Empresa", "Razão Social", "Setor", "Subsetor", "Segmento"
    dados_basicos_table = None
    for table in all_tables:
        # Verificar se algum dos labels chave está nesta tabela
        labels_in_table = [td.get_text(strip=True).lower() for td in table.find_all('td', limit=10)] # Primeiras células de label
        if any(key.lower() in labels_in_table for key in ["empresa", "razão social", "setor", "subsetor", "segmento"]):
            dados_basicos_table = table
            # logging.debug("Tabela de 'Dados Básicos da Empresa' potencialmente identificada.")
            break # Encontrou a tabela mais provável

    if not dados_basicos_table:
        # Se não encontrou pela heurística, pode tentar iterar todas ou usar a primeira,
        # mas isso é menos confiável. Para este caso, é melhor falhar se não achar a tabela certa.
        logging.warning("Não foi possível identificar a tabela principal de 'Dados Básicos da Empresa'.")
        return dados_encontrados # Retorna vazio se não achou a tabela

    # Agora processa apenas a tabela identificada
    for tr in dados_basicos_table.find_all('tr'):
        cells = tr.find_all('td')
        if len(cells) >= 2:
            label_text = cells[0].get_text(strip=True)
            
            if label_text in labels_map: # Verifica se o label atual é um dos que queremos
                output_key = labels_map[label_text]
                value_cell = cells[1]
                link_tag = value_cell.find('a')
                value_text = ""
                if link_tag:
                    value_text = link_tag.get_text(strip=True)
                else:
                    value_text = value_cell.get_text(strip=True)
                
                dados_encontrados[output_key] = value_text
                # logging.debug(f"  Encontrado na tabela específica: Label='{label_text}', Valor='{value_text}' -> Mapeado para '{output_key}'")

    return dados_encontrados


def buscar_dados_empresa_investsite(codigo_negociacao):
    """
    Busca Razão Social, Setor, Subsetor e Segmento da página de indicadores principais.
    Retorna um dicionário com os dados encontrados.
    """
    if not codigo_negociacao or pd.isna(codigo_negociacao):
        logging.warning("buscar_dados_empresa_investsite: Código de negociação inválido ou ausente.")
        return None

    codigo_fmt = str(codigo_negociacao).strip().upper()
    url = f"https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={codigo_fmt}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }

    logging.info(f"--- Buscando dados completos para {codigo_fmt} ---")
    logging.info(f"URL: {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=20) 
        logging.info(f"Status Code da Resposta: {response.status_code}")

        # DESCOMENTE PARA SALVAR HTML PARA UM CÓDIGO ESPECÍFICO
        # if codigo_fmt == "ABEV3": # Adicione os códigos que quer depurar
        #    filename_html = f"debug_html_indicadores_{codigo_fmt}.html"
        #    with open(filename_html, "w", encoding="utf-8") as f_html:
        #        f_html.write(response.text)
        #    logging.info(f"HTML salvo para {codigo_fmt} em {filename_html}")

        if response.status_code == 404:
             logging.warning(f"Página não encontrada (404) para {codigo_fmt} na URL: {url}")
             return None
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.content, 'lxml')
        
        # Mapeamento dos labels do site para as chaves do nosso dicionário de output
        labels_a_extrair = {
            "Razão Social": "nome",
            "Setor": "setor_c",
            "Subsetor": "setor_d",
            "Segmento": "setor_e" # Este é o "Segmento" que acompanha Setor/Subsetor
        }
        
        dados_extraidos = extract_data_from_specific_table(soup, labels_a_extrair)

        # Fallback para nome se "Razão Social" não for encontrada
        if not dados_extraidos.get('nome'):
            h1_tag = soup.find('h1')
            if h1_tag:
                h1_text = h1_tag.get_text(separator=" ", strip=True)
                if h1_text:
                    nome_fantasia = re.sub(r'\(' + re.escape(codigo_fmt) + r'\)', '', h1_text, flags=re.IGNORECASE).strip()
                    nome_fantasia = re.sub(r'\b' + re.escape(codigo_fmt) + r'\b', '', nome_fantasia, flags=re.IGNORECASE).strip()
                    if nome_fantasia:
                        dados_extraidos['nome'] = nome_fantasia
                        logging.info(f"Usando nome fantasia do H1 como fallback: '{nome_fantasia}' para {codigo_fmt}")
        
        logging.info(f"Dados extraídos para {codigo_fmt}: {dados_extraidos}")

        if dados_extraidos.get('nome'): # Considera sucesso se pelo menos o nome foi encontrado
            return dados_extraidos
        else:
            logging.warning(f"Nenhum nome (Razão Social ou H1) encontrado para {codigo_fmt}.")
            return None
            
    except requests.exceptions.HTTPError as e:
        logging.error(f"Erro HTTP ao buscar dados completos para {codigo_fmt} (URL: {url}): {e}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro de requisição ao buscar dados completos para {codigo_fmt} (URL: {url}): {e}")
    except Exception as e:
        logging.error(f"Erro inesperado ao processar dados completos para {codigo_fmt} (URL: {url}): {e}")
    
    logging.warning(f"FINAL: Dados completos NÃO encontrados para {codigo_fmt}.")
    return None


def main():
    print("Iniciando processo de coleta e preenchimento de informações de empresas...")

    # FORÇAR LIMPEZA DE CACHE PARA TESTE
    if os.path.exists(ARQUIVO_FINAL_COMPLETO):
        try:
            os.remove(ARQUIVO_FINAL_COMPLETO)
            print(f"Arquivo cache '{ARQUIVO_FINAL_COMPLETO}' REMOVIDO para forçar nova busca.")
        except Exception as e_rem:
            print(f"Não foi possível remover o arquivo cache '{ARQUIVO_FINAL_COMPLETO}': {e_rem}")


    codigos_ref_anterior = carregar_e_desmembrar_codigos(ARQUIVO_ANTERIOR, COLUNA_CODIGO_ANTERIOR)
    codigos_ref_atual = carregar_e_desmembrar_codigos(ARQUIVO_ATUAL, COLUNA_CODIGO_ATUAL)
    
    todos_codigos_referencia = sorted(list(codigos_ref_anterior.union(codigos_ref_atual)))
    
    if not todos_codigos_referencia:
        print("Nenhum código de referência encontrado. Abortando.")
        return
    
    print(f"Total de {len(todos_codigos_referencia)} códigos de negociação únicos para processar.")

    dados_finais_lista = [] 
    # O cache não será usado nesta execução devido à remoção acima, mas a estrutura está aqui.
    # nomes_pre_existentes_map = {} 
    # try:
    #     # ... (lógica de carregar cache, se não for deletado)
    # except FileNotFoundError:
    #     logging.info(f"Arquivo cache '{ARQUIVO_FINAL_COMPLETO}' não encontrado. Todos os dados serão buscados online.")
    # # ...

    dados_coletados_online = 0
    erros_busca_total = 0

    for i, codigo in enumerate(todos_codigos_referencia):
        logging.info(f"--- Processando {i+1}/{len(todos_codigos_referencia)}: {codigo} ---")
        
        dados_empresa_final = {
            COLUNA_CODIGO_FINAL: codigo,
            COLUNA_NOME_FINAL: pd.NA,
            COLUNA_SETOR_C: pd.NA,
            COLUNA_SETOR_D: pd.NA,
            COLUNA_SETOR_E: pd.NA
        }
        
        dados_online = buscar_dados_empresa_investsite(codigo)
        if dados_online and dados_online.get('nome'):
            dados_empresa_final[COLUNA_NOME_FINAL] = dados_online.get('nome', pd.NA)
            dados_empresa_final[COLUNA_SETOR_C] = dados_online.get('setor_c', pd.NA) # Mapeado de "Setor"
            dados_empresa_final[COLUNA_SETOR_D] = dados_online.get('setor_d', pd.NA) # Mapeado de "Subsetor"
            dados_empresa_final[COLUNA_SETOR_E] = dados_online.get('setor_e', pd.NA) # Mapeado de "Segmento"
            
            logging.info(f"Dados para {codigo}: Nome='{dados_empresa_final[COLUNA_NOME_FINAL]}', "
                         f"{COLUNA_SETOR_C}='{dados_empresa_final[COLUNA_SETOR_C]}', "
                         f"{COLUNA_SETOR_D}='{dados_empresa_final[COLUNA_SETOR_D]}', "
                         f"{COLUNA_SETOR_E}='{dados_empresa_final[COLUNA_SETOR_E]}'")
            dados_coletados_online += 1
        else:
            logging.warning(f"Não foi possível obter dados online válidos (com nome) para {codigo}.")
            erros_busca_total += 1
        
        time.sleep(1.5) 
        
        dados_finais_lista.append(dados_empresa_final)

    df_final = pd.DataFrame(dados_finais_lista)
    
    try:
        df_final.to_excel(ARQUIVO_FINAL_COMPLETO, index=False)
        print(f"\nPlanilha final gerada com sucesso em: {ARQUIVO_FINAL_COMPLETO}")
        print("--- Resumo da Geração ---")
        print(f"Total de códigos processados: {len(todos_codigos_referencia)}")
        print(f"Registros onde dados foram coletados online nesta execução: {dados_coletados_online}")
        print(f"Códigos para os quais não foi possível obter nome: {erros_busca_total}")
        print(f"Total de registros na planilha final: {len(df_final)}")

    except Exception as e:
        print(f"ERRO ao salvar a planilha final '{ARQUIVO_FINAL_COMPLETO}': {e}")

if __name__ == "__main__":
    main()

2025-05-20 16:50:30,788 - INFO - Carregados e desmembrados 385 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_anterior.xlsx' da coluna 'cod_negociacao'.
2025-05-20 16:50:30,880 - INFO - Carregados e desmembrados 382 códigos únicos de 'C:\Users\Heber Castro\Downloads\dados_novos_atual.xlsx' da coluna 'cod_negociacao'.


Iniciando processo de coleta e preenchimento de informações de empresas...


2025-05-20 16:50:30,880 - INFO - --- Processando 1/400: AALR3 ---
2025-05-20 16:50:30,880 - INFO - --- Buscando dados completos para AALR3 ---
2025-05-20 16:50:30,880 - INFO - URL: https://www.investsite.com.br/principais_indicadores.php?cod_negociacao=AALR3


Total de 400 códigos de negociação únicos para processar.


2025-05-20 16:50:31,986 - INFO - Status Code da Resposta: 200
2025-05-20 16:50:32,026 - INFO - Dados extraídos para AALR3: {'nome': 'ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.', 'setor_c': 'Saúde', 'setor_d': 'Serv.Méd.Hospit..Análises e Diagnósticos', 'setor_e': 'Serv.Méd.Hospit..Análises e Diagnósticos'}
2025-05-20 16:50:32,026 - INFO - Dados para AALR3: Nome='ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.', Setor_Atuacao='Saúde', Setor_Atuacao2='Serv.Méd.Hospit..Análises e Diagnósticos', Setor_Atuacao3='Serv.Méd.Hospit..Análises e Diagnósticos'
2025-05-20 16:50:33,530 - INFO - --- Processando 2/400: ABCB4 ---
2025-05-20 16:50:33,530 - INFO - --- Buscando dados completos para ABCB4 ---
2025-05-20 16:50:33,534 - INFO - URL: https://www.investsite.com.br/principais_indicadores.php?cod_negociacao=ABCB4
2025-05-20 16:50:34,694 - INFO - Status Code da Resposta: 200
2025-05-20 16:50:34,730 - INFO - Dados extraídos para ABCB4: {'nome': 'BCO ABC BRASIL S.A.', 'setor_c': 'Financeiro', 'setor_d': 'Intermediário


Planilha final gerada com sucesso em: C:\Users\Heber Castro\Downloads\Informacoes_Empresas_FINAL_CORRETO.xlsx
--- Resumo da Geração ---
Total de códigos processados: 400
Registros onde dados foram coletados online nesta execução: 360
Códigos para os quais não foi possível obter nome: 40
Total de registros na planilha final: 400
